In [47]:
import os
import seaborn as sns
import nibabel as nib
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

In [48]:
def softmax(x, T):
    x = x / T
    f = np.exp(x - np.max(x, axis = 0))  # shift values
    return f / f.sum(axis = 0)

def ComputMetric(ACTUAL, PREDICTED):
    ACTUAL = ACTUAL.flatten()
    PREDICTED = PREDICTED.flatten()
    idxp = ACTUAL == True
    idxn = ACTUAL == False

    tp = np.sum(ACTUAL[idxp] == PREDICTED[idxp])
    tn = np.sum(ACTUAL[idxn] == PREDICTED[idxn])
    fp = np.sum(idxn) - tn
    fn = np.sum(idxp) - tp
    FPR = fp / (fp + tn)
    if tp == 0 :
        dice = 0
        Precision = 0
        Sensitivity = 0
    else:
        dice = 2 * tp / (2 * tp + fp + fn)
        Precision = tp / (tp + fp)
        Sensitivity = tp / (tp + fn)
    return dice, Sensitivity, Precision

In [49]:
def sum_tensor(inp, axes, keepdim=False):
    axes = np.unique(axes).astype(int)
    if keepdim:
        for ax in axes:
            inp = inp.sum(int(ax), keepdim=True)
    else:
        for ax in sorted(axes, reverse=True):
            inp = inp.sum(int(ax))
    return inp

def get_tp_fp_fn(net_output, gt, axes=None, mask=None, square=False):
    """
    net_output must be (b, c, x, y(, z)))
    gt must be a label map (shape (b, 1, x, y(, z)) OR shape (b, x, y(, z))) or one hot encoding (b, c, x, y(, z))
    if mask is provided it must have shape (b, 1, x, y(, z)))
    :param net_output:
    :param gt:
    :param axes:
    :param mask: mask must be 1 for valid pixels and 0 for invalid pixels
    :param square: if True then fp, tp and fn will be squared before summation
    :return:
    """
    if axes is None:
        axes = tuple(range(2, len(net_output.size())))

    shp_x = net_output.shape
    shp_y = gt.shape

    with torch.no_grad():
        if len(shp_x) != len(shp_y):
            gt = gt.view((shp_y[0], 1, *shp_y[1:]))

        if all([i == j for i, j in zip(net_output.shape, gt.shape)]):
            # if this is the case then gt is probably already a one hot encoding
            y_onehot = gt
        else:
            gt = gt.long()
            y_onehot = torch.zeros(shp_x)
            if net_output.device.type == "cuda":
                y_onehot = y_onehot.cuda(net_output.device.index)
            y_onehot.scatter_(1, gt, 1)

    tp = net_output * y_onehot
    fp = net_output * (1 - y_onehot)
    fn = (1 - net_output) * y_onehot

    if mask is not None:
        tp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(tp, dim=1)), dim=1)
        fp = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fp, dim=1)), dim=1)
        fn = torch.stack(tuple(x_i * mask[:, 0] for x_i in torch.unbind(fn, dim=1)), dim=1)

    if square:
        tp = tp ** 2
        fp = fp ** 2
        fn = fn ** 2

    tp = sum_tensor(tp, axes, keepdim=False)
    fp = sum_tensor(fp, axes, keepdim=False)
    fn = sum_tensor(fn, axes, keepdim=False)

    return tp, fp, fn


def SoftDiceLoss(x, y, loss_mask = None, smooth = 1e-5, batch_dice = False):
    '''
    Batch_dice means that we want to calculate the dsc of all batch
    It would make more sense for small patchsize, aka DeepMedic based training.
    '''
    shp_x = x.shape
    square = False

    axes = [0] + list(range(2, len(shp_x)))
    tp, fp, fn = get_tp_fp_fn(x, y, axes, loss_mask, square)
    dc = (2 * tp + smooth) / (2 * tp + fp + fn + smooth)
    dc_process = dc

    return dc_process

In [58]:
resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostateval/'
prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletestB/'

In [59]:
DatafiletsImgc1 = prostatevalpath + 'seg-eval.txt'
Imgfiletsc1 = open(DatafiletsImgc1)
Imgreadc1 = Imgfiletsc1.read().splitlines()

In [6]:
probs_all = []
preds_class_all = []
targets_all = []
# probs_all_c0 = []
# preds_class_all_c0 = []
# targets_all_c0 = []
# probs_all_c1 = []
# preds_class_all_c1 = []
# targets_all_c1 = []
softDSCs = []
realDSCs = []
for Imgnamec1 in tqdm(Imgreadc1):
    knamelist = Imgnamec1.split("/")
    kname = knamelist[-1][0:6]
    
    cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
    cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
    cls0read = nib.load(cls0filename)
    cls1read = nib.load(cls1filename)
    cls0logit = cls0read.get_fdata()
    cls1logit = cls1read.get_fdata()
    GTread = nib.load(Imgnamec1)
    GTimg = GTread.get_fdata()
    imgshape = GTimg.shape
    cls0flatten = cls0logit.flatten()
    cls1flatten = cls1logit.flatten()
    clsflatten = np.stack((cls0flatten, cls1flatten))
    GTflatten = GTimg.flatten()
    probflatten = softmax(clsflatten, T = 1.0)
    
    pred_class = np.argmax(probflatten, axis = 0)
    preds_class_all = np.concatenate((preds_class_all, pred_class), axis=0)
    targets_all = np.concatenate((targets_all, GTflatten), axis=0)
    probmax = np.max(probflatten, axis = 0)
    probs_all = np.concatenate((probs_all, probmax), axis=0)

    preds_all_argmax = np.argmax(clsflatten, axis = 0)
    
#     # for cls 0, BG class 
#     targets_y1 = np.where(preds_all_argmax==0)[0]
#     probflatten_c0 = softmax(clsflatten[:, targets_y1], T = 1.0)
#     pred_class_c0 = np.argmax(probflatten_c0, axis = 0)
#     preds_class_all_c0 = np.concatenate((preds_class_all_c0, pred_class_c0), axis=0)
    
#     targets_all_c0 = np.concatenate((targets_all_c0, GTflatten[targets_y1]), axis=0)
#     probmax_c0 = np.max(probflatten_c0, axis = 0)
#     probs_all_c0 = np.concatenate((probs_all_c0, probmax_c0), axis=0)
    
#     # for cls 1, FG class
#     targets_y1 = np.where(preds_all_argmax==1)[0]
#     probflatten_c1 = softmax(clsflatten[:, targets_y1], T = 1.0)
#     pred_class_c1 = np.argmax(probflatten_c1, axis = 0)
#     preds_class_all_c1 = np.concatenate((preds_class_all_c1, pred_class_c1), axis=0)
    
#     targets_all_c1 = np.concatenate((targets_all_c1, GTflatten[targets_y1]), axis=0)
#     probmax_c1 = np.max(probflatten_c1, axis = 0)
#     probs_all_c1 = np.concatenate((probs_all_c1, probmax_c1), axis=0)

    # for cls 0, BG class 
    targets_y1 = np.where(preds_all_argmax==0)[0]
    probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
    # for cls 1, FG class
    targets_y1 = np.where(preds_all_argmax==1)[0]
    probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
    probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
    probr_tensor = torch.tensor(probr[np.newaxis, ...])
    GTimgf = np.argmax(probr, axis = 0)
    GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
    
    softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
    softDSCs.append(softDSC[1].numpy())

    realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
    realDSCs.append(realDSC)

acc = np.sum(targets_all == preds_class_all) / len(targets_all)
mean_prob = np.mean(probs_all)
# acc_c0 = np.sum(targets_all_c0 == preds_class_all_c0) / len(targets_all_c0)
# mean_prob_c0 = np.mean(probs_all_c0)
# acc_c1 = np.sum(targets_all_c1 == preds_class_all_c1) / len(targets_all_c1)
# mean_prob_c1 = np.mean(probs_all_c1)
print(acc)
print(mean_prob)
# print(acc_c0)
# print(mean_prob_c0)
# print(acc_c1)
# print(mean_prob_c1)
msoftDSC = np.mean(softDSCs)
mrealDSC = np.mean(realDSCs)
print(msoftDSC)
print(mrealDSC)

100%|█████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.36s/it]

0.9960332029006206
0.9983169080179795
0.9590364072630848
0.8925794874268455


In [60]:
# 84 conditions
softDSCs_AC = []
softDSCs_DoC = []
softDSCs_CSDoC = []
realDSCs = []
for kcon in tqdm(range(1, 84)):
    softDSC_FG_AC = []
    softDSC_FG_DoC = []
    softDSC_FG_CSDoC = []
    realDSC_FG = []
    resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostattestcondition_' + str(kcon) + '/'
    for Imgnamec1 in Imgreadc1:
        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
        cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
        cls0read = nib.load(cls0filename)
        cls1read = nib.load(cls1filename)
        cls0logit = cls0read.get_fdata()
        cls1logit = cls1read.get_fdata()
        GTread = nib.load(Imgnamec1)
        GTimg = GTread.get_fdata()
        imgshape = GTimg.shape
        cls0flatten = cls0logit.flatten()
        cls1flatten = cls1logit.flatten()
        clsflatten = np.stack((cls0flatten, cls1flatten))
        GTflatten = GTimg.flatten()
        # By AC
        probflatten = softmax(clsflatten, T = 1.0)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_AC.append(softDSC[1].numpy())
        # By DoC
        probflattens = softmax(clsflatten, T = 1.0)
        # calculate the diff
        
        preds_all_argmax = np.argmax(clsflatten, axis = 0)

        probflattens[0, np.where(preds_all_argmax == 0)] = probflattens[0, np.where(preds_all_argmax == 0)] + (acc - mean_prob)
        probflattens[1, np.where(preds_all_argmax == 0)] = probflattens[1, np.where(preds_all_argmax == 0)] - (acc - mean_prob)
        probflattens[0, np.where((1-preds_all_argmax) == 0)] = probflattens[0, np.where((1-preds_all_argmax) == 0)] - (acc - mean_prob)
        probflattens[1, np.where((1-preds_all_argmax) == 0)] = probflattens[1, np.where((1-preds_all_argmax) == 0)] + (acc - mean_prob)
        probflattens = np.clip(probflattens, 0, 1)
        
#         probflatten = probflatten + (acc - mean_prob)
        
        probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probrs_tensor = torch.tensor(probrs[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probrs_tensor, GT_tensor)
        softDSC_FG_DoC.append(softDSC[1].numpy())
        # By CSDoC
        preds_all_argmax = np.argmax(clsflatten, axis = 0)
        # for cls 0, BG class 
        targets_y1 = np.where(preds_all_argmax==0)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        
#         probflatten[:, targets_y1] =  probflatten[:, targets_y1] + (acc_c0 - mean_prob_c0)
        
        # for cls 1, FG class
        targets_y1 = np.where(preds_all_argmax==1)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        
#         probflatten[:, targets_y1] = probflatten[:, targets_y1] + (acc_c1 - mean_prob_c1)
        probflatten = np.clip(probflatten, 0, 1)
        
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_CSDoC.append(softDSC[1].numpy() + (mrealDSC - msoftDSC))

        realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
        realDSC_FG.append(realDSC)
        
    softDSCs_AC.append(np.mean(np.array(softDSC_FG_AC)))
    softDSCs_DoC.append(np.mean(np.array(softDSC_FG_DoC)))
    softDSCs_CSDoC.append(np.mean(np.array(softDSC_FG_CSDoC)))
    realDSCs.append(np.mean(np.array(realDSC_FG)))

100%|█████████████████████████████████████████████████████████████████| 83/83 [19:04<00:00, 13.78s/it]


In [61]:
print('AC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_AC))))
print('DoC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_DoC))))
print('CSDoC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_CSDoC))))

AC_results:
0.08682389850061455
DoC_results:
0.03653038206070081
CSDoC_results:
0.03495739062819712


In [62]:
print(realDSCs)

[0.8457301025307677, 0.870852084028978, 0.828521585768393, 0.790703746725564, 0.7586812862096527, 0.7557484783931, 0.8172327417988589, 0.6604643235449457, 0.8391672071049449, 0.629645911710675, 0.8541350026276865, 0.8736883469998509, 0.8538392221470215, 0.843655497285545, 0.8472068280655369, 0.8415826376847552, 0.8719698924620335, 0.8669337478504607, 0.8188883956186596, 0.8289616381345647, 0.7512113795312175, 0.7984512037339606, 0.8258757147417111, 0.7808367447866433, 0.8216387475687809, 0.7407959047695183, 0.8082450924634974, 0.7488012437878321, 0.8813964100757584, 0.8449041600923041, 0.8723101712065781, 0.7438437175087971, 0.7690050430372952, 0.8036237249343758, 0.6054180714311662, 0.594775255065044, 0.8655259284098629, 0.8193493642947048, 0.8471036691597765, 0.8195559172815617, 0.8942332191332024, 0.886877812107558, 0.8949294033819303, 0.8655104773584853, 0.8947390346757661, 0.8509641068080747, 0.8916376091423593, 0.8932068281575198, 0.8886573243066662, 0.894043014237129, 0.88414099

In [63]:
print(softDSCs_AC)

[0.9445208141411179, 0.5600351926555271, 0.931730127079436, 0.6528295465927425, 0.9158539060827395, 0.7136134168357384, 0.9246074382786308, 0.6644763539333387, 0.9389497001221697, 0.7066098338452236, 0.9275612026510706, 0.9325569574687153, 0.9328646004932912, 0.9269801658772193, 0.9270234060337872, 0.9238830251708064, 0.9326770866378464, 0.9336464948378751, 0.9172174173486736, 0.9205507685038764, 0.9035785475003083, 0.9033867852464283, 0.7677991683602265, 0.77648857500991, 0.7670021114181449, 0.7822310609348627, 0.7681708070921458, 0.7782648221080334, 0.9547345456520384, 0.9456113594412106, 0.9535450027183474, 0.9004201920300152, 0.8979303404342277, 0.9448680173310867, 0.8859718451761012, 0.8879338421392813, 0.9540584116065987, 0.9288409827503431, 0.9366174510538441, 0.9459088235601767, 0.9619089739592205, 0.9532975605133911, 0.9635967070577143, 0.9466395431589174, 0.9636146188932045, 0.9455708263766514, 0.9578893452463093, 0.9599190227327112, 0.9549463403474387, 0.961062565393781, 0.9

In [64]:
print(softDSCs_DoC)

[0.8855793806178365, 0.5398399096207067, 0.875723285609153, 0.6284892613416123, 0.8686629839093885, 0.6853645574281281, 0.8697781731540773, 0.6450446712007141, 0.8773593294892106, 0.6851297361285489, 0.8690729434979731, 0.8716239451820261, 0.8747628745997265, 0.8710270977740052, 0.8692966026258009, 0.868128389865036, 0.8712530114340925, 0.8724938084921778, 0.8625001705690561, 0.8661448880030557, 0.8571648318697497, 0.8544912742744298, 0.7312085073487132, 0.7424442267119907, 0.7313407603111631, 0.7506173807240824, 0.7338308957764645, 0.7465327423151089, 0.8909439375054158, 0.8874454825765292, 0.8913197916837626, 0.8534444904768034, 0.8481614815341201, 0.8902813174619297, 0.8519269213637914, 0.8540081804680268, 0.8919466201288913, 0.8725604987282631, 0.8781731100484649, 0.890714832390233, 0.8960767468057249, 0.8894387725404238, 0.8975101165924977, 0.8858045838342656, 0.897567442823459, 0.8862165125284106, 0.8928496801199428, 0.8944743873941732, 0.8905827447837578, 0.8954026577127088, 0.8

In [65]:
print(softDSCs_CSDoC)

[0.8780638943048785, 0.4935782728192878, 0.8652732072431965, 0.586372626756503, 0.8493969862465004, 0.647156496999499, 0.8581505184423914, 0.5980194340970993, 0.8724927802859301, 0.6401529140089843, 0.8611042828148312, 0.8661000376324759, 0.8664076806570519, 0.86052324604098, 0.8605664861975478, 0.8574261053345671, 0.8662201668016071, 0.8671895750016357, 0.8507604975124344, 0.8540938486676373, 0.837121627664069, 0.836929865410189, 0.7013422485239872, 0.7100316551736705, 0.7005451915819055, 0.7157741410986234, 0.7017138872559064, 0.7118079022717939, 0.8882776258157989, 0.8791544396049714, 0.887088082882108, 0.833963272193776, 0.8314734205979883, 0.8784110974948474, 0.8195149253398618, 0.8214769223030419, 0.8876014917703593, 0.8623840629141037, 0.8701605312176047, 0.8794519037239376, 0.895452054122981, 0.8868406406771518, 0.897139787221475, 0.880182623322678, 0.8971576990569652, 0.8791139065404121, 0.8914324254100698, 0.893462102896472, 0.8884894205111994, 0.8946056455575417, 0.885060619

In [53]:
# 5 conditions
domainlist = ['A', 'C', 'D', 'E', 'F']
softDSCs_AC = []
softDSCs_DoC = []
softDSCs_CSDoC = []
realDSCs = []
for kcon in tqdm(domainlist):
    softDSC_FG_AC = []
    softDSC_FG_DoC = []
    softDSC_FG_CSDoC = []
    realDSC_FG = []
    resultpath = '/vol/biomedic3/zl9518/ModelEvaluation/output/prostate/prostattestcondition_' + kcon + '/'
    
    prostatevalpath = '/vol/biomedic3/zl9518/Prostatedata/datafiletest' + kcon + '/'
    DatafiletsImgc1 = prostatevalpath + 'seg-eval.txt'
    Imgfiletsc1 = open(DatafiletsImgc1)
    Imgreadc1 = Imgfiletsc1.read().splitlines()
    for Imgnamec1 in Imgreadc1:
        knamelist = Imgnamec1.split("/")
        kname = knamelist[-1][0:6]

        cls0filename = resultpath + '/results/pred_' + kname + 'cls0_prob.nii.gz'
        cls1filename = resultpath + '/results/pred_' + kname + 'cls1_prob.nii.gz'
        cls0read = nib.load(cls0filename)
        cls1read = nib.load(cls1filename)
        cls0logit = cls0read.get_fdata()
        cls1logit = cls1read.get_fdata()
        GTread = nib.load(Imgnamec1)
        GTimg = GTread.get_fdata()
        imgshape = GTimg.shape
        cls0flatten = cls0logit.flatten()
        cls1flatten = cls1logit.flatten()
        clsflatten = np.stack((cls0flatten, cls1flatten))
        GTflatten = GTimg.flatten()
        # By AC
        probflatten = softmax(clsflatten, T = 1.0)
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
        softDSC_FG_AC.append(softDSC[1].numpy())
        # By DoC
        probflattens = softmax(clsflatten, T = 1.0)
        # calculate the diff
        
        preds_all_argmax = np.argmax(clsflatten, axis = 0)

        probflattens[0, np.where(preds_all_argmax == 0)] = probflattens[0, np.where(preds_all_argmax == 0)] + (acc - mean_prob)
        probflattens[1, np.where(preds_all_argmax == 0)] = probflattens[1, np.where(preds_all_argmax == 0)] - (acc - mean_prob)
        probflattens[0, np.where((1-preds_all_argmax) == 0)] = probflattens[0, np.where((1-preds_all_argmax) == 0)] - (acc - mean_prob)
        probflattens[1, np.where((1-preds_all_argmax) == 0)] = probflattens[1, np.where((1-preds_all_argmax) == 0)] + (acc - mean_prob)
        probflattens = np.clip(probflattens, 0, 1)
        
#         probflatten = probflatten + (acc - mean_prob)
        
        probrs = probflattens.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probrs_tensor = torch.tensor(probrs[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probrs_tensor, GT_tensor)
        softDSC_FG_DoC.append(softDSC[1].numpy())
        # By CSDoC
        preds_all_argmax = np.argmax(clsflatten, axis = 0)
        # for cls 0, BG class 
        targets_y1 = np.where(preds_all_argmax==0)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        
#         probflatten[:, targets_y1] =  probflatten[:, targets_y1] + (acc_c0 - mean_prob_c0)
        
        # for cls 1, FG class
        targets_y1 = np.where(preds_all_argmax==1)[0]
        probflatten[:, targets_y1] = softmax(clsflatten[:, targets_y1], T = 1.0)
        
#         probflatten[:, targets_y1] = probflatten[:, targets_y1] + (acc_c1 - mean_prob_c1)
        probflatten = np.clip(probflatten, 0, 1)
        
        probr = probflatten.reshape((2, imgshape[0], imgshape[1], imgshape[2]))
        probr_tensor = torch.tensor(probr[np.newaxis, ...])
        GTimgf = np.argmax(probr, axis = 0)
        GT_tensor = torch.tensor(GTimgf[np.newaxis, ...])
        softDSC = SoftDiceLoss(probr_tensor, GT_tensor)
#         softDSC_FG_CSDoC.append(softDSC[1].numpy())
        softDSC_FG_CSDoC.append(softDSC[1].numpy() + (mrealDSC - msoftDSC))

        realDSC, _, _ = ComputMetric(GTimg, np.argmax(probr, axis = 0))
        realDSC_FG.append(realDSC)
        
    softDSCs_AC.append(np.mean(np.array(softDSC_FG_AC)))
    softDSCs_DoC.append(np.mean(np.array(softDSC_FG_DoC)))
    softDSCs_CSDoC.append(np.mean(np.array(softDSC_FG_CSDoC)))
    realDSCs.append(np.mean(np.array(realDSC_FG)))

100%|███████████████████████████████████████████████████████████████████| 5/5 [00:40<00:00,  8.04s/it]


In [54]:
print('AC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_AC))))
print('DoC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_DoC))))
print('CSDoC_results:')
print(np.mean(np.abs(np.array(realDSCs) - np.array(softDSCs_CSDoC))))

AC_results:
0.18734415411981173
DoC_results:
0.13932456190498504
CSDoC_results:
0.12088723428357233


In [55]:
print(softDSCs_AC)

[0.8903888833710243, 0.8646945640943136, 0.8701191490312503, 0.9339760285393177, 0.9073143222637996]


In [56]:
print(softDSCs_DoC)

[0.8672925405702486, 0.8179851444913133, 0.8173482835437254, 0.881894540373737, 0.8418744772465476]


In [57]:
print(softDSCs_CSDoC)

[0.8239319635347849, 0.7982376442580741, 0.8036622291950108, 0.8675191087030785, 0.8408574024275601]
